# Agno Agents with Nebius AI and Couchbase




In this notebook, we'll explore:
1. Setting up a simple agent with web search capability
2. Creating an agent with a knowledge base for specialized domains
3. Building multi-agent systems where specialized agents work together

## Install required packages for this notebook
- agno: The agent framework we'll be using
- duckduckgo-search: For web search capabilities
- pypdf: For processing PDF documents  
- Couchbase: For vector storage
- yfinance: For financial data retrieval

In [ ]:
!pip install -qU agno duckduckgo-search pypdf couchbase yfinance openai

## Simple Agent

Below we create a basic agent that can search the web to answer questions. This agent uses:
- meta-llama/Llama-3.3-70B-Instruct
- DuckDuckGo search as a tool to retrieve information from the web

When we run this agent with a question about the NBA, it will search for recent information and provide an answer.

In [ ]:
from agno.agent import Agent
from agno.models.nebius import Nebius
from agno.tools.duckduckgo import DuckDuckGoTools

agent = Agent(
    model=Nebius(
        id="meta-llama/Llama-3.3-70B-Instruct",
        api_key=os.getenv("NEBIUS_API_KEY")
    ),
    tools=[DuckDuckGoTools()],
    markdown=True
)

agent.print_response("How are the golden state warriors doing this 2024-2025 season?", stream=True)

Output()

# Agent with Knowledge Base

This example demonstrates how to create an agent with specialized knowledge. We'll build a Thai cuisine expert by:
1. Loading a PDF containing Thai recipes into a knowledge base
2. Setting up a vector database to store and retrieve this information efficiently
3. Configuring the agent to prioritize its knowledge base while still being able to search the web

This type of agent is ideal for specialized domains where you want to combine proprietary knowledge with the ability to find supplementary information.

In [ ]:
import os
# Import additional components for knowledge base functionality
from agno.agent import Agent
from agno.models.nebius import Nebius
from agno.embedder.openai import OpenAIEmbedder
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase
from agno.vectordb.couchbase import CouchbaseSearch
from couchbase.options import ClusterOptions, KnownConfigProfiles
from couchbase.auth import PasswordAuthenticator


# Couchbase connection settings
username = os.getenv("COUCHBASE_USER", "")
password = os.getenv("COUCHBASE_PASSWORD", "")
connection_string = os.getenv("COUCHBASE_CONNECTION_STRING", "")

# Create cluster options with authentication
auth = PasswordAuthenticator(username, password)
cluster_options = ClusterOptions(auth)
cluster_options.apply_profile(KnownConfigProfiles.WanDevelopment)

# Create an agent with a specialized knowledge base for Thai recipes
agent = Agent(
    model=Nebius(
        id="meta-llama/Llama-3.3-70B-Instruct",
        api_key=os.getenv("NEBIUS_API_KEY")
        ),
    # Define the agent's role
    description="You are a Thai cuisine expert!",
    # Give the agent specific instructions on how to use its knowledge
    instructions=[
        "Search your knowledge base for Thai recipes.",
        "If the question is better suited for the web, search the web to fill in gaps.",
        "Prefer the information in your knowledge base over the web results."
    ],
    # Set up the knowledge base from a PDF URL
    knowledge=PDFUrlKnowledgeBase(
        urls=["https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf"],
        # Configure the vector database for semantic search
        vector_db=CouchbaseSearch(
        bucket_name="recipe_bucket",
        scope_name="recipe_scope",
        collection_name="recipes",
        couchbase_connection_string=connection_string,
        cluster_options=cluster_options,
        search_index="vector_search_fts_index",
        embedder=OpenAIEmbedder(
            id="text-embedding-3-large",
            dimensions=3072,
            api_key=os.getenv("OPENAI_API_KEY")
        ),
        wait_until_index_ready=60,
        overwrite=True
    ),
    ),
    # Add web search capability as a backup
    tools=[DuckDuckGoTools()],
    # Display tool usage in the output
    show_tool_calls=True,
    markdown=True
)



# Load the knowledge base (only needs to be done once)
if agent.knowledge is not None:
    agent.knowledge.load()

agent.print_response("How do I make chicken and galangal in coconut milk soup", stream=True)
agent.print_response("What is the history of Thai curry?", stream=True)

INFO Connected to Couchbase successfully.

INFO Loading knowledge base

INFO Reading: https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf

INFO Skipped 14 existing/duplicate documents.

INFO Added 0 documents to knowledge base

Output()

Output()

# Multi-Agent System

This example demonstrates how to create a team of specialized agents that work together. We'll build:
1. A web search agent for finding general information
2. A finance agent for retrieving financial data
3. A coordinator agent that delegates tasks to the specialized agents

This approach allows us to create more powerful systems by combining specialized capabilities.

In [ ]:
from agno.agent import Agent
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.models.nebius import Nebius
from agno.tools.yfinance import YFinanceTools

# Create a specialized agent for web searches
web_agent = Agent(
    name="Web Agent",
    role="Search the web for information",
    # Using the larger Llama 3.3 (70B) model for better performance
    model=Nebius(
        id="meta-llama/Llama-3.3-70B-Instruct",
        api_key=os.getenv("NEBIUS_API_KEY")
    ),
    tools=[DuckDuckGoTools()],
    instructions="Always include sources",
    show_tool_calls=True,
    markdown=True,
)

# Create a specialized agent for financial data
finance_agent = Agent(
    name="Finance Agent",
    role="Get financial data",
    model=Nebius(
        id="meta-llama/Llama-3.3-70B-Instruct",
        api_key=os.getenv("NEBIUS_API_KEY")
    ),
    # Financial tools for stock data, analyst recommendations, and company info
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True, company_info=True)],
    instructions="Use tables to display data",
    show_tool_calls=True,
    markdown=True,
)

# Create a coordinator agent that manages the team
agent_team = Agent(
    # Provide the specialized agents as a team
    team=[web_agent, finance_agent],
    # The coordinator also uses a powerful model
    model=Nebius(
        id="meta-llama/Llama-3.3-70B-Instruct",
        api_key=os.getenv("NEBIUS_API_KEY")
    ),
    # Instructions for the final output
    instructions=["Always include sources", "Use tables to display data"],
    show_tool_calls=True,
    markdown=True,
)

agent_team.print_response("What's the market outlook and financial performance of AI semiconductor companies?", stream=True)

Output()